In [220]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import time
import re

# PRIMERO ABRE LA PAGINA PRINCIPAL, EL EMPIZA A BAJAR CON SCROLL, HASTA LLEGAR AL FINAL DE LA PAGINA, UNA VES ESTO REALIZA LA CAPTURA DE TODA LA PAGINA

In [3]:
#chrome_options = Options()
#driver = webdriver.Chrome()



chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

url = 'https://henki.com.mx/producto/results/categoria/1'
driver.get(url)
time.sleep(1.5)

bloques2a=[]
bloques3a=[]
chains = ActionChains(driver)
for a in range(401): # escroll de la pagina hasta bajar a final de la pagina, imprime cuantos scroll va realizando con pausas para permitir que la pagina cargue
    chains.scroll_by_amount(0, 900).perform()
    time.sleep(1.5)
    print(f'Scroll {a} de 400', end = '\r')
    
sopa = BeautifulSoup(driver.page_source)
bloques2a = sopa.select('div.col-md div div div.items-buttons a[href]') #captura todos los link de cada producto
bloques3a = sopa.select('div.items-child div a.text-decoration-none.text-reset') #captura todos los nombres de cada producto

In [4]:
len(bloques2a) # cantidad de link y nombvres capturados

2397

# 2397 PRODUCTOS , CONTAMOS CON SU NOMBRE Y LINK DE PAGINA PRINCIAPAL DE CADA PRODUCTO

In [5]:
lista_link=[]
for item in bloques2a:
    lista_link.append(item['href']) # todos los links, los guarda en una lista 

In [ ]:
lista_nombre=[]
for item in bloques3a:
    lista_nombre.append(item.text.strip('\n ')) # todos los nombres, los guarda en una lista 

# AHORA ABRIMOS TODOS LOS LINK DE LOS PRODUCTOS, PARA PODER REALIZAR UN REQUEST Y OBETENER LA INFORMACION DESEADA
# EN ESTE CASO ES SACAR LA MARCA, NOMBRE, DESCRIPCION DE PRODUCTO, MODELO Y URL DE LA IMAGEN DEL PRODUCTO

In [6]:
lista_marca=[]
lista_nombre2=[]
lista_descripcion=[]
lista_modelo=[]
lista_img_url=[]
lista_precio=[]
a=0

for link in lista_link: # FOR PARA ENTRAR A TODOS LOS LINK DE PRODUCTOS
    url_link =link
    response_link = requests.get(url_link) # REQUEST DE LA PAGINA
    sopa_git_link = BeautifulSoup(response_link.content, 'html5lib') 
    
    #MARCA --- OBTENCION DE LA MARCA DEL PRODUCTO Y LIMPIEZA
    descripcion_link= sopa_git_link.select('div.col-12.col-md-4.item-descripcion.align-self-start.text-justify')
    texto= descripcion_link[0].text
    texto2 = re.search("(Marca:)(.+)(\n)", texto)
    lista_marca.append(texto2.group(2))
    
    #selecion tabla (DEBIDO A QUE DENTRO TIENE LA MAYORIA DE LA INFORMACION)
    descripcion_nombre= sopa_git_link.select('table.table.table-bordered.table-striped tbody tr td.w-50') 
    
    #OBTENEMOS EL NOMBRE O TITULO DEL PRODUCTO 
    lista_nombre2.append(descripcion_nombre[0].text)
    #OBTENEMOS LA DESCRIPCION DEL PRODUCTO
    lista_descripcion.append(descripcion_nombre[1].text) 
    #OBTENEMOS EL MODELO 
    lista_modelo.append(descripcion_nombre[4].text) 

    #OBTENEMOS EL LNK DE LA IMAGEN DEL PRODUCTO
    descripcion_img_url = sopa_git_link.select('div.img-container.text-center a[data-fancybox] img')
    lista_img_url.append(descripcion_img_url[0]['src'])
    #url imagen
    
    
    #OBTENEMOS EL PRECIO DEL PRODUCTO
    try :
        lista_precio.append(sopa_git_link.select('div.item-price')[0].text.strip('\n '))
    except:#EN CASO DE QUE NO CUENTE CON PRECIO SE AGREGA "" , NADA
        lista_precio.append('')
    #lista_precio.append(sopa_git_link.select('div.item-price')[0].text.strip('\n '))
    #precio
    
    #IMPRIME EL AVANCE DE LA CAPTURA DE CADA PAGINA
    a=a+1
    print(f'Extrayendo pagina {a} de {len(lista_link)}', end = '\r')
    #if a==100:
    #    break

# GUARDAMOS TODOS LOS DATOS EN UNA LISTA GENERAL, PARA PODER REALIZAR UN DATA FRAME DE TODOS LOS DATOS Y MOSTRAMOS ESTE DATAFRAME Y GUARDAMOS EN UNA TABLA EXCELE

In [7]:
lista_tabla = list(zip(lista_nombre2, lista_modelo, lista_marca, lista_precio, lista_descripcion, lista_img_url))

In [8]:
df = pd.DataFrame(lista_tabla, columns=['Nombre','Modelo','Marca','Precio','Descripcion','Url IMG'])


In [9]:
df.head(2396)

,Nombre,Modelo,Marca,Precio,Descripcion,Url IMG
0,FOLDER TAMAÑO CARTA COLOR CREMA ECOLOGICO,11417,BEROKY 1,$1.52,FOLDER TAMAÑO CARTA EN CARTULINA DE 175 GRAMOS...,https://henki.com.mx/erp/shared_resources/prod...
1,SOBRE BOLSA SOLAPA ENGOMADA MANILA CARTA,SOBMGK-CTA,KIP 1,$2.27,SOBRE BOLSA SOLAPA ENGOMADA PAPEL MANILA DE 90...,https://henki.com.mx/erp/shared_resources/prod...
2,SOBRE BLANCO PARA CORRESPONDENCIA CARTA SIN VE...,SOBBCO-CTA,KIP 1,$0.56,SOBRE BLANCO PARA CORRESPONDENCIA CARTA 6 3/4 ...,https://henki.com.mx/erp/shared_resources/prod...
3,SOBRE BOLSA SOLAPA ENGOMADA MANILA EXTRA OFICIO,SOBMGK-OFI,KIP 1,$2.85,SOBRE BOLSA SOLAPA ENGOMADA PAPEL MANILA DE 90...,https://henki.com.mx/erp/shared_resources/prod...
4,BOLIGRAFO PUNTO MEDIANO AZUL,BOLEX,BOLEX 1,$2.13,BOLIGRAFO PUNTO MEDIANO AZUL BOLEX,https://henki.com.mx/erp/shared_resources/rest...
...,...,...,...,...,...,...
2391,ETIQUETA BLANCA TRANSFER,EAB01338,TRANSFER 1,$11.69,ETIQUETAS BLANCAS TRANSFER MEDIDA 13 MM X 38 M...,https://henki.com.mx/erp/shared_resources/prod...
2392,MICA TERMICA TRANSFER MODELO MTR0806595 8 MIL...,MTR0806595,TRANSFER 1,$115.22,MICA TERMICA TRANSFER MODELO MTR0806595 8 MIL...,https://henki.com.mx/erp/shared_resources/prod...
2393,ETIQUETAS FLUORESCENTES NARANJA LESA,N°19,LESA 1,$33.96,ETIQUETAS FLUORESCENTES NARANJA RECTANGULAR LE...,https://henki.com.mx/erp/shared_resources/prod...
2394,SEPARADOR KIP 10 DIVISIONES SIN NUMERO,SEKIMP-10,KIP 1,$24.79,SEPARADOR KIP 10 DIVISIONES MODELO SEKIMP-10 C...,https://henki.com.mx/erp/shared_resources/prod...


In [10]:
df.to_excel('tabla_productos.xlsx') #GUARDAMOS EN UNA TABLA DE EXCELE

In [11]:
lista_img_url[0] # PROBAMOS UN LINK DE LA LISTA URL IMG

'https://henki.com.mx/erp/shared_resources/productos/64NOIMAGEN.png'

# EL SIGUIENTE PROGRAMA REALIZA UN REQUEST DE TODAS LOS LINK DE IMAGEN, Y EXTRAER LA IMAGEN LO GUARDA EN LA CAPRETA IMAGENES Y LOS RENOMBRA A IMANGE{No}

In [27]:
# PROGRAMA PARA ENTRAR EN TODOS LOS LINK DE LA LISTA DE LINK IMAGENES Y EXTRAER LAS IMAGENES, E IRLAS GUARDANDO EN LA CAPETA IMAGENES, Y RENOMBRANDOLAS DEPENDIENDO SU NUMERO 

import PIL
from PIL import Image
import urllib.request
import requests


opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36')]
urllib.request.install_opener(opener)

a=0

for imag in lista_img_url:
    a=a+1
    imag=imag.replace(' ','%20')# en los link que tiene espacio, los llena con un %20 para que pueda reconocer el link html
    try:
        urllib.request.urlretrieve(imag, f"Imagenes/imagen{a}.png") # GUARDANDO LAS IMAGENES Y RENOMBRANDO LAS COMO IMAGEN{NUMERO de contador} 
    except:
        urllib.request.urlretrieve('https://henki.com.mx/erp/shared_resources/productos/64NOIMAGEN.png', f"Imagenes/imagen{a}.png") #EN EL CASO DE QUE TENGA ERROR EL LINK DE IMAGENES JALAR UNA DEFINIDA
    #img = PIL.Image.open(f"Imagenes/imagen{a}.png")
    print(f'Extrayendo pagina {a} de {len(lista_img_url)}', end = '\r') #imprime como va el proceso de obtencion de imagenes
    #img.show()

# AHORA QUE TENEMOS LAS IMAGENES GUARDADOS, VAMOS A CARGR LAS IMAGENES , VAMOS A CREEAR UN NUEVO EXCEL, PERO AHORA PEGANDOLE LAS IMAGENES EN CADA CELDA CORRESPONDIENTE

In [32]:
import pandas as pd


# Create a Pandas dataframe from some data.
#df = pd.DataFrame({'Data': [10, 20, 30, 20, 15, 30, 45]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Tabla_producto_image.xlsx', engine='xlsxwriter') # RENOMBRANDO EL NUEVO EXCELN COMO TABLA_PRODUCTO_IMAGE

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1') # ACTIVAMOS EL PODER RESCRIBIR EL LIBRO SHEET1

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1'] # CREAMOS LA FUNCION DE SOBRE ESCIRBIR 

# Insert an image.
#worksheet.insert_image('I2', 'logo.png')
b=2

for imag in range(len(lista_img_url)):
    imag=imag+1
    worksheet.insert_image(f'I{b}', f'Imagenes/imagen{imag}.png') # CARGAMOS LA IMAGEN NUMERADA Y LA PEGAMOS EN LA CELDA I NUMERADA
    b=b+1


# Close the Pandas Excel writer and output the Excel file.
writer.save() # GUARDAMOS EL EXCEL

# LIMPIEZA DE LA TABLA

In [227]:
df = pd.read_excel('tabla_productos.xlsx', index_col=0) 

In [228]:
df.head(5)

,Nombre,Modelo,Marca,Precio,Descripcion,Url IMG
0,FOLDER TAMAÑO CARTA COLOR CREMA ECOLOGICO,11417,BEROKY 1,$1.52,FOLDER TAMAÑO CARTA EN CARTULINA DE 175 GRAMOS...,https://henki.com.mx/erp/shared_resources/prod...
1,SOBRE BOLSA SOLAPA ENGOMADA MANILA CARTA,SOBMGK-CTA,KIP 1,$2.27,SOBRE BOLSA SOLAPA ENGOMADA PAPEL MANILA DE 90...,https://henki.com.mx/erp/shared_resources/prod...
2,SOBRE BLANCO PARA CORRESPONDENCIA CARTA SIN VE...,SOBBCO-CTA,KIP 1,$0.56,SOBRE BLANCO PARA CORRESPONDENCIA CARTA 6 3/4 ...,https://henki.com.mx/erp/shared_resources/prod...
3,SOBRE BOLSA SOLAPA ENGOMADA MANILA EXTRA OFICIO,SOBMGK-OFI,KIP 1,$2.85,SOBRE BOLSA SOLAPA ENGOMADA PAPEL MANILA DE 90...,https://henki.com.mx/erp/shared_resources/prod...
4,BOLIGRAFO PUNTO MEDIANO AZUL,BOLEX,BOLEX 1,$2.13,BOLIGRAFO PUNTO MEDIANO AZUL BOLEX,https://henki.com.mx/erp/shared_resources/rest...


In [229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2397 entries, 0 to 2396
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Nombre       2397 non-null   object
 1   Modelo       2380 non-null   object
 2   Marca        2397 non-null   object
 3   Precio       2381 non-null   object
 4   Descripcion  2397 non-null   object
 5   Url IMG      2397 non-null   object
dtypes: object(6)
memory usage: 131.1+ KB


In [230]:
df.isnull().sum()

Nombre          0
Modelo         17
Marca           0
Precio         16
Descripcion     0
Url IMG         0
dtype: int64

In [231]:
df['Modelo'].fillna('Sin Modelo', inplace = True)

In [232]:
df.isnull().sum()

Nombre          0
Modelo          0
Marca           0
Precio         16
Descripcion     0
Url IMG         0
dtype: int64

In [233]:
df['Precio'].fillna(0, inplace = True)

In [234]:
df.isnull().sum()

Nombre         0
Modelo         0
Marca          0
Precio         0
Descripcion    0
Url IMG        0
dtype: int64

In [235]:
df.Marca.unique()

array(['BEROKY        1', 'KIP        1', 'BOLEX        1',
       'JOCAR        1', 'MYP        1', 'AZOR        1',
       'OXFORD        1', 'SIN MARCA        1', 'HERCULES        1',
       'ESCRIMEX        1', 'APSA        1', 'BACO        1',
       'KYMA        1', 'A-INK        1', 'FORTEC        1',
       'MAE        1', 'DIXON        1', 'LEADER        1',
       'SABONIS        1', 'BEROL        1', 'JYL        1',
       'ACCO        1', 'BIC        1', 'DIEM        1', 'TROSA        1',
       'BUFFY        3          ', 'GBC        1', 'ESTRELLA        1',
       'MAPASA        1', 'PELIKAN        1', 'MEMO TIP        1',
       'WILSON JONES        1', 'MANNY        1', 'BARRILITO        1',
       'SHELY        1', 'CORTY        1', 'PENDAFLEX        1',
       'HENKEL        1', 'KYMA        3          ',
       'FLASHFILE        3          ', 'JANEL        1', 'KORES        1',
       'INDUX        1', 'OKAMI        1', 'PETIGON        1',
       'STEMPEL        1', 

In [236]:
df.Marca = df.Marca.str.replace(r'\s+\d', '', regex = True).str.strip(' ')

In [237]:
df.Marca.unique()

array(['BEROKY', 'KIP', 'BOLEX', 'JOCAR', 'MYP', 'AZOR', 'OXFORD',
       'SIN MARCA', 'HERCULES', 'ESCRIMEX', 'APSA', 'BACO', 'KYMA',
       'A-INK', 'FORTEC', 'MAE', 'DIXON', 'LEADER', 'SABONIS', 'BEROL',
       'JYL', 'ACCO', 'BIC', 'DIEM', 'TROSA', 'BUFFY', 'GBC', 'ESTRELLA',
       'MAPASA', 'PELIKAN', 'MEMO TIP', 'WILSON JONES', 'MANNY',
       'BARRILITO', 'SHELY', 'CORTY', 'PENDAFLEX', 'HENKEL', 'FLASHFILE',
       'JANEL', 'KORES', 'INDUX', 'OKAMI', 'PETIGON', 'STEMPEL',
       'ARTLINE', 'PCM', 'PILOT', 'RICMEN', 'OFFICIUM', 'COPAMEX',
       'PEGASO', 'FACTIS', '3M', 'PRITT', 'FIFA', 'RESISTOL', 'OLIVIA',
       'GOODY LASER DE MEXICO', 'LAZZARI', 'PRODUCTOS GM', 'PENTEL',
       'VEROMO', 'EUROCOLOR', 'TODO DE CARTON', 'PINOS ALTOS',
       'SWINGLINE', 'LEFORT', 'RAYTER', 'KINERA', 'EL CUERVO', 'ZEBRA',
       'LESA', 'SCRIBE', 'TUK', 'HUERTA', 'BOFLEX', 'PONTOR', 'MIL´U',
       'LAFICA', 'IRIS', 'TLG', 'PEGA QUIP', 'KOLA LOKA', 'FORMATA',
       'EPSON', 'ACEGRAPAS FIFA'

In [238]:
df.Precio.unique()

array(['$1.52', '$2.27', '$0.56', ..., '$18.30', '$115.22', '$302.06'],
      dtype=object)

In [239]:
df.Precio = df.Precio.str.strip('$').str.replace(',', '')

In [240]:
df.head(2)

,Nombre,Modelo,Marca,Precio,Descripcion,Url IMG
0,FOLDER TAMAÑO CARTA COLOR CREMA ECOLOGICO,11417,BEROKY,1.52,FOLDER TAMAÑO CARTA EN CARTULINA DE 175 GRAMOS...,https://henki.com.mx/erp/shared_resources/prod...
1,SOBRE BOLSA SOLAPA ENGOMADA MANILA CARTA,SOBMGK-CTA,KIP,2.27,SOBRE BOLSA SOLAPA ENGOMADA PAPEL MANILA DE 90...,https://henki.com.mx/erp/shared_resources/prod...


In [241]:
df.dtypes

Nombre         object
Modelo         object
Marca          object
Precio         object
Descripcion    object
Url IMG        object
dtype: object

In [242]:
df.Precio

0         1.52
1         2.27
2         0.56
3         2.85
4         2.13
         ...  
2392    115.22
2393     33.96
2394     24.79
2395    302.06
2396     47.95
Name: Precio, Length: 2397, dtype: object

In [243]:
df.Precio = df.Precio.astype('float')

In [244]:
df.head(5)

,Nombre,Modelo,Marca,Precio,Descripcion,Url IMG
0,FOLDER TAMAÑO CARTA COLOR CREMA ECOLOGICO,11417,BEROKY,1.52,FOLDER TAMAÑO CARTA EN CARTULINA DE 175 GRAMOS...,https://henki.com.mx/erp/shared_resources/prod...
1,SOBRE BOLSA SOLAPA ENGOMADA MANILA CARTA,SOBMGK-CTA,KIP,2.27,SOBRE BOLSA SOLAPA ENGOMADA PAPEL MANILA DE 90...,https://henki.com.mx/erp/shared_resources/prod...
2,SOBRE BLANCO PARA CORRESPONDENCIA CARTA SIN VE...,SOBBCO-CTA,KIP,0.56,SOBRE BLANCO PARA CORRESPONDENCIA CARTA 6 3/4 ...,https://henki.com.mx/erp/shared_resources/prod...
3,SOBRE BOLSA SOLAPA ENGOMADA MANILA EXTRA OFICIO,SOBMGK-OFI,KIP,2.85,SOBRE BOLSA SOLAPA ENGOMADA PAPEL MANILA DE 90...,https://henki.com.mx/erp/shared_resources/prod...
4,BOLIGRAFO PUNTO MEDIANO AZUL,BOLEX,BOLEX,2.13,BOLIGRAFO PUNTO MEDIANO AZUL BOLEX,https://henki.com.mx/erp/shared_resources/rest...


In [245]:
df.dtypes

Nombre          object
Modelo          object
Marca           object
Precio         float64
Descripcion     object
Url IMG         object
dtype: object

In [246]:
df.to_csv('tabla_productos_limpio.cvs', index=False) #GUARDAMOS EN UNA TABLA CVS

In [247]:
df.to_excel('tabla_productos_limpio.xlsx', index=False) #GUARDAMOS EN UNA TABLA DE EXCEL

In [248]:
df.insert(6, "IMAGEN", "") 

In [249]:
df.head(2)

,Nombre,Modelo,Marca,Precio,Descripcion,Url IMG,IMAGEN
0,FOLDER TAMAÑO CARTA COLOR CREMA ECOLOGICO,11417,BEROKY,1.52,FOLDER TAMAÑO CARTA EN CARTULINA DE 175 GRAMOS...,https://henki.com.mx/erp/shared_resources/prod...,
1,SOBRE BOLSA SOLAPA ENGOMADA MANILA CARTA,SOBMGK-CTA,KIP,2.27,SOBRE BOLSA SOLAPA ENGOMADA PAPEL MANILA DE 90...,https://henki.com.mx/erp/shared_resources/prod...,


In [250]:
len(df)

2397

In [252]:
import pandas as pd


# Create a Pandas dataframe from some data.
#df = pd.DataFrame({'Data': [10, 20, 30, 20, 15, 30, 45]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Tabla_producto_image_limpio.xlsx', engine='xlsxwriter') # RENOMBRANDO EL NUEVO EXCELN COMO TABLA_PRODUCTO_IMAGE

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, index=False, sheet_name='Sheet1') # ACTIVAMOS EL PODER RESCRIBIR EL LIBRO SHEET1

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1'] # CREAMOS LA FUNCION DE SOBRE ESCIRBIR 

# Insert an image.
#worksheet.insert_image('I2', 'logo.png')
b=2

for imag in range(len(df)):
    imag=imag+1
    worksheet.insert_image(f'G{b}', f'Imagenes/imagen{imag}.png') # CARGAMOS LA IMAGEN NUMERADA Y LA PEGAMOS EN LA CELDA I NUMERADA
    b=b+1


# Close the Pandas Excel writer and output the Excel file.
writer.save() # GUARDAMOS EL EXCEL

In [223]:
help(df.to_excel)

Help on method to_excel in module pandas.core.generic:

to_excel(excel_writer, sheet_name: 'str' = 'Sheet1', na_rep: 'str' = '', float_format: 'str | None' = None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True, freeze_panes=None, storage_options: 'StorageOptions' = None) -> 'None' method of pandas.core.frame.DataFrame instance
    Write object to an Excel sheet.
    
    To write a single object to an Excel .xlsx file it is only necessary to
    specify a target file name. To write to multiple sheets it is necessary to
    create an `ExcelWriter` object with a target file name, and specify a sheet
    in the file to write to.
    
    Multiple sheets may be written to by specifying unique `sheet_name`.
    With all data written to the file it is necessary to save the changes.
    Note that creating an `ExcelWriter` object with a file name that already
    exists will result in t